In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract
import sqlite3

In [2]:
books_df = pd.read_csv("Resources/Nilim-Kaggle/books.csv", error_bad_lines=False)
books_df.dropna(inplace = True) 
books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
3346,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,674842111,9.78E+12,en-US,236,61,6,4/20/2004,Harvard University Press
4700,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,9.78E+12,eng,400,26,4,4/6/2004,Cold Spring Press
5875,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,9.78E+12,eng,342,38,4,1/15/1999,Huntington House Publishers
8952,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0,851742718,9.78E+12,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.


In [3]:
books_df["publisher"].value_counts()

4/6/2004     1
4/20/2004    1
5/1/1977     1
1/15/1999    1
Name: publisher, dtype: int64

In [4]:
books_df["language_code"].value_counts()

9.78E+12    4
Name: language_code, dtype: int64

In [5]:
#en-CA = English (Canada), en-GB =English (United Kingdom), en-US= English (United States) update all to eng. 
eng_books_df= books_df.loc[(books_df["language_code"] == "en-CA") | (books_df["language_code"] == "en-GB")| (books_df["language_code"] == "en-US")].replace(['en-CA','en-GB','en-US'],'eng')
eng_books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12


In [6]:
books_df["language_code"].value_counts()

9.78E+12    4
Name: language_code, dtype: int64

In [7]:
books_df.dtypes

bookID                 int64
title                 object
authors               object
average_rating        object
isbn                  object
isbn13                object
language_code         object
  num_pages           object
ratings_count          int64
text_reviews_count     int64
publication_date      object
publisher             object
Unnamed: 12           object
dtype: object

In [8]:
books_df['publication_date'] = pd.to_datetime(books_df['publication_date'], dayfirst=True, errors='coerce')

In [9]:
books_df.dtypes

bookID                         int64
title                         object
authors                       object
average_rating                object
isbn                          object
isbn13                        object
language_code                 object
  num_pages                   object
ratings_count                  int64
text_reviews_count             int64
publication_date      datetime64[ns]
publisher                     object
Unnamed: 12                   object
dtype: object

##### Database Setup 

In [10]:
conn = sqlite3.connect('books_test_db')
c = conn.cursor()

In [11]:
c.execute('''CREATE TABLE books(bookID, title, authors, average_rating, isbn, isbn13, language_code, num_pages, ratings_count, text_reviews_count, publication_date, publisher)''')
conn.commit()

In [12]:
books_df.to_sql('books', conn, if_exists='replace', index = False)

C:\Users\nmlam\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [13]:
engine = create_engine("sqlite:///books_test_db.sqlite")

In [14]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)